# Glassdoor Analysis

In [2]:
import pandas as pd

#numerical library for python
import numpy as np

#headless browser
# go to anaconda prompt and enter: pip install selenium
from selenium import webdriver

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#make pc not do anything for certain amount of time
from time import sleep

import re

# Load library
from nltk.corpus import stopwords


import nltk
nltk.download('wordnet')

import nltk
nltk.download('stopwords')

import nltk
from nltk.stem import WordNetLemmatizer 

from matplotlib import pyplot as plt

#datetime package
from datetime import datetime

import numpy as np


# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nirun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nirun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\nirun\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df = pd.read_csv('glassdoor_full_dataset.csv')
df['date'] = pd.to_datetime(df.date)
df['unique_id'] = df.index
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 20 columns):
date                         703 non-null datetime64[ns]
star                         703 non-null int64
subject                      702 non-null object
pros                         703 non-null object
cons                         703 non-null object
advice                       394 non-null object
location                     408 non-null object
clean_member_type            702 non-null object
tenure                       703 non-null object
employee_status              703 non-null object
recommend                    520 non-null object
outlook                      512 non-null object
ceo                          505 non-null object
Career Opportunities         621 non-null object
Compensation and Benefits    620 non-null object
Culture & Values             609 non-null object
Senior Management            601 non-null object
Work/Life Balance            621 non-null object
star_g

## New columns with symbols removed

In [4]:
df['sw_pros'] = df.pros.fillna('none')
df['sw_pros'] = df['sw_pros'].map(lambda x: re.sub('[^a-zA-Z]', ' ', x))
df['sw_pros'] =df['sw_pros'].map(lambda x: re.sub(r'\s+', ' ', x))

df['sw_cons'] = df.cons.fillna('none')
df['sw_cons'] = df['sw_cons'].map(lambda x: re.sub('[^a-zA-Z]', ' ', x))
df['sw_cons'] =df['sw_cons'].map(lambda x: re.sub(r'\s+', ' ', x))


## New columns with stopwords removed

In [5]:
# Load stop words
stop = stopwords.words('english')
newStopWords = ['A','also','can', 'Can', 'i', 'dedpends', 'go' ,'too','if','I', 'can', 'it', 'not' , 
                'nt', 'get', 'you','worked','not','if','lot','per','even','nt','enough','it','always',
                'there','need','lots','part','can','bit','last','things','thing','put','make',"a", "about", 
                "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", 
                "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", 
                "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", 
                "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", 
                "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", 
                "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", 
                "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", 
                "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", 
                "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", 
                "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", 
                "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", 
                "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", 
                "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", 
                "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", 
                "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", 
                "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", 
                "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", 
                "we've", "what's", "when's", "where's", "who's", "why's", "would",'de','la','the', 'day', 'la', 'like',
                'customers','really','en','Manger','que','customer','every','many','It','El','one','much','You',
                'well','un','Sometimes','never','una','los','give','If','everyone','want','They','There','week',
                'Everything','n','empresa','di','first','know','weekly','keep','del','co','Lo','es','il','por','era',
                "a",	"about",	"above",	"after",	"again",	"against",	"all",	"am",	"an",	"and",	"any",	"are",	"aren't",	"as",	"at",	"be",	"because",
                "been",	"before",	"being",	"below",	"between",	"both",	"but",	"by",	"can't",	"cannot",	"could",	"couldn't",	"did",	"didn't",	"do",	"does",	"doesn't",
                "doing",	"don't",	"down",	"during",	"each",	"few",	"for",	"from",	"further",	"had",	"hadn't",	"has",	"hasn't",	"have",	"haven't",	"having",	"he",
                "he'd",	"he'll",	"he's",	"her",	"here",	"here's",	"hers",	"herself",	"him",	"himself",	"his",	"how",	"how's",	"i",	"i'd",	"i'll",	"i'm",
                "i've",	"if",	"in",	"into",	"is",	"isn't",	"it",	"it's",	"its",	"itself",	"let's",	"me",	"more",	"most",	"mustn't",	"my",	"myself",
                "no",	"nor",	"not",	"of",	"off",	"on",	"once",	"only",	"or",	"other",	"ought",	"our",	"ours",	"ourselves",	"out",	"over",	"own",
                "same",	"shan't",	"she",	"she'd",	"she'll",	"she's",	"should",	"shouldn't",	"so",	"some",	"such",	"than",	"that",	"that's",	"the",	"their",	"theirs",
                "them",	"themselves",	"then",	"there",	"there's",	"these",	"they",	"they'd",	"they'll",	"they're",	"they've",	"this",	"those",	"through",	"to",	"too",	"under",
                "until",	"up",	"very",	"was",	"wasn't",	"we",	"we'd",	"we'll",	"we're",	"we've",	"were",	"weren't",	"what",	"what's",	"when",	"when's",	"where",
                "where's",	"which",	"while",	"who",	"who's",	"whom",	"why",	"why's",	"with",	"won't",	"would",	"wouldn't",	"you",	"you'd",	"you'll",	"you're",	"you've",
                "your",	"yours",	"yourself",	"yourselve",	"a's",	"accordingly",	"again",	"allows",	"also",	"amongst",	"anybody",	"anyways",	"appropriate",	"aside",	"available",	"because",	"before",
                "below",	"between",	"by",	"can't",	"certain",	"com",	"consider",	"corresponding",	"definitely",	"different",	"don't",	"each",	"else",	"et",	"everybody",	"exactly",	"fifth",
                "follows",	"four",	"gets",	"goes",	"greetings",	"has",	"he",	"her",	"herein",	"him",	"how",	"i'm",	"immediate",	"indicate",	"instead",	"it",	"itself",
                "know",	"later",	"lest",	"likely",	"ltd",	"me",	"more",	"must",	"nd",	"needs",	"next",	"none",	"nothing",	"of",	"okay",	"ones",	"others",
                "ourselves",	"own",	"placed",	"probably",	"rather",	"regarding",	"right",	"saying",	"seeing",	"seen",	"serious",	"she",	"so",	"something",	"soon",	"still",	"t's",
                "th",	"that",	"theirs",	"there",	"therein",	"they'd",	"third",	"though",	"thus",	"toward",	"try",	"under",	"unto",	"used",	"value",	"vs",	"way",
                "we've",	"weren't",	"whence",	"whereas",	"whether",	"who's",	"why",	"within",	"wouldn't",	"you'll",	"yourself",	"able",	"across",	"against",	"almost",	"although",	"an",
                "anyhow",	"anywhere",	"are",	"ask",	"away",	"become",	"beforehand",	"beside",	"beyond",	"c'mon",	"cannot",	"certainly",	"come",	"considering",	"could",	"described",	"do",
                "done",	"edu",	"elsewhere",	"etc",	"everyone",	"example",	"first",	"for",	"from",	"getting",	"going",	"had",	"hasn't",	"he's",	"here",	"hereupon",	"himself",
                "howbeit",	"i've",	"in",	"indicated",	"into",	"it'd",	"just",	"known",	"latter",	"let",	"little",	"mainly",	"mean",	"moreover",	"my",	"near",	"neither",
                "nine",	"noone",	"novel",	"off",	"old",	"only",	"otherwise",	"out",	"particular",	"please",	"provides",	"rd",	"regardless",	"said",	"says",	"seem",	"self",
                "seriously",	"should",	"some",	"sometime",	"sorry",	"sub",	"take",	"than",	"that's",	"them",	"there's",	"theres",	"they'll",	"this",	"three",	"to",	"towards",
                "trying",	"unfortunately",	"up",	"useful",	"various",	"want",	"we",	"welcome",	"what",	"whenever",	"whereby",	"which",	"whoever",	"will",	"without",	"yes",	"you're",
                "yourselves",	"about",	"actually",	"ain't",	"alone",	"always",	"and",	"anyone",	"apart",	"aren't",	"asking",	"awfully",	"becomes",	"behind",	"besides",	"both",	"c's",
                "cant",	"changes",	"comes",	"contain",	"couldn't",	"despite",	"does",	"down",	"eg",	"enough",	"even",	"everything",	"except",	"five",	"former",	"further",	"given",
                "gone",	"hadn't",	"have",	"hello",	"here's",	"hers",	"his",	"however",	"ie",	"inasmuch",	"indicates",	"inward",	"it'll",	"keep",	"knows",	"latterly",	"let's",
                "look",	"many",	"meanwhile",	"most",	"myself",	"nearly",	"never",	"no",	"nor",	"now",	"often",	"on",	"onto",	"ought",	"outside",	"particularly",	"plus",
                "que",	"re",	"regards",	"same",	"second",	"seemed",	"selves",	"seven",	"shouldn't",	"somebody",	"sometimes",	"specified",	"such",	"taken",	"thank",	"thats",	"themselves",
                "thereafter",	"thereupon",	"they're",	"thorough",	"through",	"together",	"tried",	"twice",	"unless",	"upon",	"uses",	"very",	"wants",	"we'd",	"well",	"what's",	"where",
                "wherein",	"while",	"whole",	"willing",	"won't",	"yet",	"you've",	"zero",	"above",	"after",	"all",	"along",	"am",	"another",	"anything",	"appear",	"around",
                "associated",	"be",	"becoming",	"being",	"best",	"brief",	"came",	"cause",	"clearly",	"concerning",	"containing",	"course",	"did",	"doesn't",	"downwards",	"eight",	"entirely",
                "ever",	"everywhere",	"far",	"followed",	"formerly",	"furthermore",	"gives",	"got",	"happens",	"haven't",	"help",	"hereafter",	"herself",	"hither",	"i'd",	"if",	"inc",
                "inner",	"is",	"it's",	"keeps",	"last",	"least",	"like",	"looking",	"may",	"merely",	"mostly",	"name",	"necessary",	"nevertheless",	"nobody",	"normally",	"nowhere",
                "oh",	"once",	"or",	"our",	"over",	"per",	"possible",	"quite",	"really",	"relatively",	"saw",	"secondly",	"seeming",	"sensible",	"several",	"since",	"somehow",
                "somewhat",	"specify",	"sup",	"tell",	"thanks",	"the",	"then",	"thereby",	"these",	"they've",	"thoroughly",	"throughout",	"too",	"tries",	"two",	"unlikely",	"us",
                "using",	"via",	"was",	"we'll",	"went",	"whatever",	"where's",	"whereupon",	"whither",	"whom",	"wish",	"wonder",	"you",	"your",	"according",	"afterwards",	"allow",
                "already",	"among",	"any",	"anyway",	"appreciate",	"as",	"at",	"became",	"been",	"believe",	"better",	"but",	"can",	"causes",	"co",	"consequently",	"contains",
                "currently",	"didn't",	"doing",	"during",	"either",	"especially",	"every",	"ex",	"few",	"following",	"forth",	"get",	"go",	"gotten",	"hardly",	"having",	"hence",
                "hereby",	"hi",	"hopefully",	"i'll",	"ignored",	"indeed",	"insofar",	"isn't",	"its",	"kept",	"lately",	"less",	"liked",	"looks",	"maybe",	"might",	"much",
                "namely",	"need",	"new",	"non",	"not",	"obviously",	"ok",	"one",	"other",	"ours",	"overall",	"perhaps",	"presumably",	"qv",	"reasonably",	"respectively",	"say",
                "see",	"seems",	"sent",	"shall",	"six",	"someone",	"somewhere",	"specifying",	"sure",	"tends",	"thanx",	"their",	"thence",	"therefore",	"they",	"think",	"those",
                "thru",	"took",	"truly",	"un",	"until",	"use",	"usually",	"viz",	"wasn't",	"we're",	"were",	"when",	"whereafter",	"wherever",	"who",	"whose",	"with",
                "would",	"you'd",	"yours",	"I",	"a",	"about",	"an",	"are",	"as",	"at",	"be",	"by",	"com",	"for",	"from",	"how",	"where",
                "who",	"will",	"with",	"the",	"www",	"in",	"is",	"it",	"of",	"on",	"or",	"that",	"the",	"this",	"to",	"was",	"what",
                "when",	"a",	"able",	"about",	"above",	"abst",	"accordance",	"according",	"accordingly",	"across",	"act",	"actually",	"added",	"adj",	"affected",	"affecting",	"affects",
                "after",	"afterwards",	"again",	"against",	"ah",	"all",	"almost",	"alone",	"along",	"already",	"also",	"although",	"always",	"am",	"among",	"amongst",	"an",
                "and",	"announce",	"another",	"any",	"anybody",	"anyhow",	"anymore",	"anyone",	"anything",	"anyway",	"anyways",	"anywhere",	"apparently",	"approximately",	"are",	"aren",	"arent",
                "arise",	"around",	"as",	"aside",	"ask",	"asking",	"at",	"auth",	"available",	"away",	"awfully",	"b",	"back",	"be",	"became",	"because",	"become",
                "becomes",	"becoming",	"been",	"before",	"beforehand",	"begin",	"beginning",	"beginnings",	"begins",	"behind",	"being",	"believe",	"below",	"beside",	"besides",	"between",	"beyond",
                "biol",	"both",	"brief",	"briefly",	"but",	"by",	"c",	"ca",	"came",	"can",	"cannot",	"can't",	"cause",	"causes",	"certain",	"certainly",	"co",
                "com",	"come",	"comes",	"contain",	"containing",	"contains",	"could",	"couldnt",	"d",	"date",	"did",	"didn't",	"different",	"do",	"does",	"doesn't",	"doing",
                "done",	"don't",	"down",	"downwards",	"due",	"during",	"e",	"each",	"ed",	"edu",	"effect",	"eg",	"eight",	"eighty",	"either",	"else",	"elsewhere",
                "end",	"ending",	"enough",	"especially",	"et",	"et-al",	"etc",	"even",	"ever",	"every",	"everybody",	"everyone",	"everything",	"everywhere",	"ex",	"except",	"f",
                "far",	"few",	"ff",	"fifth",	"first",	"five",	"fix",	"followed",	"following",	"follows",	"for",	"former",	"formerly",	"forth",	"found",	"four",	"from",
                "further",	"furthermore",	"g",	"gave",	"get",	"gets",	"getting",	"give",	"given",	"gives",	"giving",	"go",	"goes",	"gone",	"got",	"gotten",	"h",
                "had",	"happens",	"hardly",	"has",	"hasn't",	"have",	"haven't",	"having",	"he",	"hed",	"hence",	"her",	"here",	"hereafter",	"hereby",	"herein",	"heres",
                "hereupon",	"hers",	"herself",	"hes",	"hi",	"hid",	"him",	"himself",	"his",	"hither",	"home",	"how",	"howbeit",	"however",	"hundred",	"i",	"id",
                "ie",	"if",	"i'll",	"im",	"immediate",	"immediately",	"importance",	"important",	"in",	"inc",	"indeed",	"index",	"information",	"instead",	"into",	"invention",	"inward",
                "is",	"isn't",	"it",	"itd",	"it'll",	"its",	"itself",	"i've",	"j",	"just",	"k",	"keep",	"keeps",	"kept",	"kg",	"km",	"know",
                "known",	"knows",	"l",	"largely",	"last",	"lately",	"later",	"latter",	"latterly",	"least",	"less",	"lest",	"let",	"lets",	"like",	"liked",	"likely",
                "line",	"little",	"'ll",	"look",	"looking",	"looks",	"ltd",	"m",	"made",	"mainly",	"make",	"makes",	"many",	"may",	"maybe",	"me",	"mean",
                "means",	"meantime",	"meanwhile",	"merely",	"mg",	"might",	"million",	"miss",	"ml",	"more",	"moreover",	"most",	"mostly",	"mr",	"mrs",	"much",	"mug",
                "must",	"my",	"myself",	"n",	"na",	"name",	"namely",	"nay",	"nd",	"near",	"nearly",	"necessarily",	"necessary",	"need",	"needs",	"neither",	"never",
                "nevertheless",	"new",	"next",	"nine",	"ninety",	"no",	"nobody",	"non",	"none",	"nonetheless",	"noone",	"nor",	"normally",	"nos",	"not",	"noted",	"nothing",
                "now",	"nowhere",	"o",	"obtain",	"obtained",	"obviously",	"of",	"off",	"often",	"oh",	"ok",	"okay",	"old",	"omitted",	"on",	"once",	"one",
                "ones",	"only",	"onto",	"or",	"ord",	"other",	"others",	"otherwise",	"ought",	"our",	"ours",	"ourselves",	"out",	"outside",	"over",	"overall",	"owing",
                "own",	"p",	"page",	"pages",	"part",	"particular",	"particularly",	"past",	"per",	"perhaps",	"placed",	"please",	"plus",	"poorly",	"possible",	"possibly",	"potentially",
                "pp",	"predominantly",	"present",	"previously",	"primarily",	"probably",	"promptly",	"proud",	"provides",	"put",	"q",	"que",	"quickly",	"quite",	"qv",	"r",	"ran",
                "rather",	"rd",	"re",	"readily",	"really",	"recent",	"recently",	"ref",	"refs",	"regarding",	"regardless",	"regards",	"related",	"relatively",	"research",	"respectively",	"resulted",
                "resulting",	"results",	"right",	"run",	"s",	"said",	"same",	"saw",	"say",	"saying",	"says",	"sec",	"section",	"see",	"seeing",	"seem",	"seemed",
                "seeming",	"seems",	"seen",	"self",	"selves",	"sent",	"seven",	"several",	"shall",	"she",	"shed",	"she'll",	"shes",	"should",	"shouldn't",	"show",	"showed",
                "shown",	"showns",	"shows",	"significant",	"significantly",	"similar",	"similarly",	"since",	"six",	"slightly",	"so",	"some",	"somebody",	"somehow",	"someone",	"somethan",	"something",
                "sometime",	"sometimes",	"somewhat",	"somewhere",	"soon",	"sorry",	"specifically",	"specified",	"specify",	"specifying",	"still",	"stop",	"strongly",	"sub",	"substantially",	"successfully",	"such",
                "sufficiently",	"suggest",	"sup",	"sure",	"t",	"take",	"taken",	"taking",	"tell",	"tends",	"th",	"than",	"thank",	"thanks",	"thanx",	"that",	"that'll",
                "thats",	"that've",	"the",	"their",	"theirs",	"them",	"themselves",	"then",	"thence",	"there",	"thereafter",	"thereby",	"thered",	"therefore",	"therein",	"there'll",	"thereof",
                "therere",	"theres",	"thereto",	"thereupon",	"there've",	"these",	"they",	"theyd",	"they'll",	"theyre",	"they've",	"think",	"this",	"those",	"thou",	"though",	"thoughh",
                "thousand",	"throug",	"through",	"throughout",	"thru",	"thus",	"til",	"tip",	"to",	"together",	"too",	"took",	"toward",	"towards",	"tried",	"tries",	"truly",
                "try",	"trying",	"ts",	"twice",	"two",	"u",	"un",	"under",	"unfortunately",	"unless",	"unlike",	"unlikely",	"until",	"unto",	"up",	"upon",	"ups",
                "us",	"use",	"used",	"useful",	"usefully",	"usefulness",	"uses",	"using",	"usually",	"v",	"value",	"various",	"'ve",	"very",	"via",	"viz",	"vol",
                "vols",	"vs",	"w",	"want",	"wants",	"was",	"wasnt",	"way",	"we",	"wed",	"welcome",	"we'll",	"went",	"were",	"werent",	"we've",	"what",
                "whatever",	"what'll",	"whats",	"when",	"whence",	"whenever",	"where",	"whereafter",	"whereas",	"whereby",	"wherein",	"wheres",	"whereupon",	"wherever",	"whether",	"which",	"while",
                "whim",	"whither",	"who",	"whod",	"whoever",	"whole",	"who'll",	"whom",	"whomever",	"whos",	"whose",	"why",	"widely",	"willing",	"wish",	"with",	"within",
                "without",	"wont",	"words",	"world",	"would",	"wouldnt",	"www",	"x",	"y",	"yes",	"yet",	"you",	"youd",	"you'll",	"your",	"youre",	"yours",
                "yourself",	"yourselves","you've","z","zero","initially","you’ll","the","1","2","3","4","5","6","7","8","9",
                "0","!","£","$","%","^","&","*","(",")","_","-","+",".",",",">","<","?","@",";",";","#","~"

                
                
                 ]
stop.extend(newStopWords)


#df['sw_subject'] = df['subject'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['sw_pros'] = df['sw_pros'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['sw_cons'] = df['sw_cons'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['sw_advice'] = df['advice'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))



df.head()

,date,star,subject,pros,cons,advice,location,clean_member_type,tenure,employee_status,...,ceo,Career Opportunities,Compensation and Benefits,Culture & Values,Senior Management,Work/Life Balance,star_group,unique_id,sw_pros,sw_cons
0,2019-11-04,5,Big dynamic company with many opportunities,Fast grows up if you are hard worker,unstable rota and early morning shifts,NaN,London,Team Leader,I have been working at Pret A Manger full-time...,Current Employee,...,NaN,Positive,Positive,Positive,Positive,Neutral,Positive,0,fast grows hard worker,unstable rota early morning shifts
1,2020-01-20,5,Great Vacation Work,Lots of shifts and flexibility of hours,Management are lax on food waste,NaN,Gatwick,Team Member,I worked at Pret A Manger full-time for less t...,Former Employee,...,NaN,Positive,Positive,Positive,Positive,Positive,Positive,1,shifts flexibility hours,management lax food waste
2,2020-01-20,4,Great company,"Great training, benefits and salary",Nothing to say so far,NaN,NaN,Team Leader,I have been working at Pret A Manger full-time,Current Employee,...,NaN,Positive,Positive,Positive,Positive,Neutral,Positive,2,great training benefits salary,
3,2020-01-16,3,Something Helpful,The worst thing is loss of bonus.,New management created silly rules and people ...,Respect your staff more.They are amazing hard ...,London,Production Team Member,I worked at Pret A Manger full-time for more t...,Former Employee,...,positive,Negative,Negative,Negative,Neutral,Negative,Neutral,3,worst loss bonus,management created silly rules people working ...
4,2020-01-15,3,One day probation,"Lot to learn, busy, new people",Very busy,Be more patient with newbies,Palmers Green,Kitchen Staff,I worked at Pret A Manger full-time for less t...,Former Employee,...,NaN,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,4,learn busy people,busy


In [6]:
df['sw_cons'] = df.apply(lambda row: nltk.word_tokenize(row['sw_cons']), axis=1)
df['sw_pros'] = df.apply(lambda row: nltk.word_tokenize(row['sw_pros']), axis=1)

lemmatizer = WordNetLemmatizer()

df['sw_cons'] = df['sw_cons'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x] ))
df['sw_pros'] = df['sw_pros'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x] ))


## Parsed word table

In [7]:
parsed_word_stage = df[['unique_id','date', 'star_group', 'sw_cons']]

# add comma between each word
parsed_word_stage = parsed_word_stage.replace(' ', ',', regex=True)


In [8]:
parsed_word_stage.head()

,unique_id,date,star_group,sw_cons
0,0,2019-11-04,Positive,"unstable,rota,early,morning,shift"
1,1,2020-01-20,Positive,"management,lax,food,waste"
2,2,2020-01-20,Positive,
3,3,2020-01-16,Neutral,"management,created,silly,rule,people,working,m..."
4,4,2020-01-15,Neutral,busy


In [32]:
# Set up empty df
master_df = pd.DataFrame()

for i, row in parsed_word_stage.iterrows():
    
    # Create a list of the con words
    con_list = row['sw_cons'].split(',')
    
    # Make df
    con_df = pd.DataFrame(con_list, columns = ['sw_cons'])
    
    # Populate extra columns
    con_df['date'] = row['date']
    con_df['star_group'] = row['star_group']
    con_df['unique_id'] = row['unique_id']
  
    master_df = master_df.append(con_df, ignore_index = True)
    
print(master_df.shape)
master_df.head()

(5200, 4)


,sw_cons,date,star_group,unique_id
0,day,2019-09-13,Positive,0
1,work,2019-12-11,Neutral,1
2,kitchen,2019-12-11,Neutral,1
3,mystery,2019-12-11,Neutral,1
4,shopper,2019-12-11,Neutral,1


In [33]:
master_df['post_march_flag'] = 0
master_df.loc[master_df.date > '20190301', 'post_march_flag'] = 1


In [34]:
master_df.to_csv('cons_index.csv', index = False)

PermissionError: [Errno 13] Permission denied: 'cons_index.csv'

## Pros

In [43]:
pros_parsed_word_stage = df[['unique_id','date', 'star_group', 'sw_pros']]

# add comma between each word
pros_parsed_word_stage = pros_parsed_word_stage.replace(' ', ',', regex=True)

In [44]:
# Set up empty df
master_df_pros = pd.DataFrame()

for i, row in pros_parsed_word_stage.iterrows():
    
    # Create a list of the con words
    pro_list = row['sw_pros'].split(',')
    
    # Make df
    pro_df = pd.DataFrame(pro_list, columns = ['sw_pros'])
    
    # Populate extra columns
    pro_df['date'] = row['date']
    pro_df['star_group'] = row['star_group']
    pro_df['unique_id'] = row['unique_id']
  
    master_df_pros = master_df_pros.append(pro_df, ignore_index = True)
    
print(master_df_pros.shape)
master_df_pros.head()

(4405, 4)


,sw_pros,date,star_group,unique_id
0,training,2019-09-13,Positive,0
1,starting,2019-09-13,Positive,0
2,salary,2019-09-13,Positive,0
3,pleasant,2019-09-13,Positive,0
4,environment,2019-09-13,Positive,0


In [45]:
master_df_pros['post_march_flag'] = 0
master_df_pros.loc[master_df_pros.date > '20190301', 'post_march_flag'] = 1

In [47]:
master_df_pros.to_csv('pros_index.csv', index = False)

## Lemmatization

In [49]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import ast

from collections import Counter

def phrase_counter(df, col, word_range, additional_stop_words = [], output_number = 100):

    # Change keyword argument
    stop_status = ENGLISH_STOP_WORDS.union(additional_stop_words)

    # Initiate the vectoriser

    # Use kwargs min_word_range, max_word_range and reference stop words
    vect = CountVectorizer(ngram_range=(word_range, word_range), stop_words=stop_status)

    # Fill with balnk spaces
    df[col] = df[col].fillna('')

    # Drop any numbers


    # Just into one big string
    summaries = ''.join(df[col])
    ngram_summaries = vect.build_analyzer()(summaries)

    # Now generate output - use output number here
    count_list = Counter(ngram_summaries).most_common(output_number)

    wordcount_dict = {'word' : [],
                     'count' : []}

    # Loop through the count_list
    for element in count_list:

        # Append the phrases and counts to dict
        wordcount_dict['word'].append(element[0])
        wordcount_dict['count'].append(element[1])

    # Output a dataframe
    count_df = pd.DataFrame(wordcount_dict)

    # Just so look pretty
    count_df = count_df[['word', 'count']]

    return count_df





In [50]:
phrase_counter(df, 'sw_cons', 2, additional_stop_words=['mystery'])

,word,count
0,team member,24
1,team leader,11
2,hard work,8
3,working hour,7
4,early morning,7
5,long hour,7
6,low pay,6
7,work pret,6
8,pret manger,5
9,work hard,5
